In [1]:
import os
import numpy as np
import tifffile as tiff
import cv2

def fft_and_linear_polar_transform(image, mask):
    # 이미지 FFT 변환
    f = np.fft.fft2(image)
    fshift = np.fft.fftshift(f)
    magnitude_spectrum = np.abs(fshift)
    
    # FFT 결과와 마스크 곱하기
    masked_spectrum = magnitude_spectrum * mask

    # FFT 이미지 크기 얻기
    height, width = masked_spectrum.shape[:2]

    # Polar 변환
    center = (width // 2, height // 2)
    max_radius = np.sqrt(center[0]**2 + center[1]**2)
    polar_image = cv2.warpPolar(masked_spectrum, (width, height), center, max_radius, cv2.WARP_FILL_OUTLIERS + cv2.WARP_POLAR_LINEAR)
    
    return polar_image

def process_tif_files(input_folder, output_folder, mask_file):
    # 마스크 파일 읽기
    mask = tiff.imread(mask_file)
    
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # 입력 폴더 내의 모든 .tif 파일에 대해 반복
    for filename in os.listdir(input_folder):
        if filename.endswith('.tif'):
            # 파일 경로 생성
            file_path = os.path.join(input_folder, filename)
            
            # .tif 파일 읽기
            image = tiff.imread(file_path)
            
            # FFT 및 Polar 변환 수행
            polar_image = fft_and_linear_polar_transform(image, mask)
            
            # 결과를 .tif 파일로 저장
            output_filename = f"polar_fft_{filename}"
            output_path = os.path.join(output_folder, output_filename)
            tiff.imwrite(output_path, polar_image.astype(np.float32))
            
            print(f'Processed {filename}, saved Polar FFT result to {output_filename}')

# 입력 폴더와 출력 폴더 경로 설정
input_folder = r"D:\Users\doyeob\Desktop\3layerBP data\devided data\atomic image\original data"
output_folder = r"D:\Users\doyeob\Desktop\3layerBP data\devided data\polar\polar data"

# 마스크 파일 경로 설정
mask_file = r"D:\Users\doyeob\Desktop\3layerBP data\mask\Big mask.tif"

# .tif 파일 처리
process_tif_files(input_folder, output_folder, mask_file)

print("All files have been processed.")


ValueError: <COMPRESSION.LZW: 5> requires the 'imagecodecs' package